In [1]:
%tensorflow_version 2.x   #For Google Colab notebook
import tensorflow as tf

print(tf.version)  

`%tensorflow_version` only switches the major version: 1.x or 2.x.
You set: `2.x   #For Google Colab notebook`. This will be interpreted as: `2.x`.


TensorFlow 2.x selected.
<module 'tensorflow._api.v2.version' from '/usr/local/lib/python3.7/dist-packages/tensorflow/_api/v2/version/__init__.py'>


In [2]:
import pandas as pd
import numpy as np

In [3]:
#Used to load the csv file into Google Colab
from google.colab import files   #https://medium.com/@simonprdhm/2-ways-to-upload-csv-files-to-google-colab-4d29ffa9db85
data_to_load = files.upload()

Saving aug_train.csv to aug_train (8).csv


In [4]:
import io
df = pd.read_csv(io.BytesIO(data_to_load['aug_train.csv']))

In [ ]:
#shows how many of each, in a graph representation (barh means bar histogram)

#df.gender.value_counts().plot(kind='barh');  #more than 10k males vs females
#df.major_discipline.value_counts().plot(kind='barh');  
#df.education_level.value_counts().plot(kind='barh'); #about 60% are graduates
#df.city.value_counts().value_counts().plot(kind='barh');  #(drop)
#df.city_development_index.value_counts().value_counts().plot(kind='pie'); #seems mostly evenly split, higher numbers mean more developed city
#df.relevent_experience.value_counts().value_counts().plot(kind='pie');

In [ ]:
#shows all the cols,  and u specify how many rows to print. We have 19,159 total rows (including headers)
print(df.head(15)) 

In [8]:
#shows the datatype for that header. Object is for data that isnt form a string
print(df['education_level'].dtypes)  

object


In [10]:
#outputs all the datatypes for all the heads

df.dtypes  

enrollee_id                 int64
city                       object
city_development_index    float64
gender                     object
relevent_experience        object
enrolled_university        object
education_level            object
major_discipline           object
experience                 object
company_size               object
company_type               object
last_new_job               object
training_hours              int64
target                      int64
dtype: object

In [5]:
#Column Dropper
 
#dropped enrolle_id, city, city_developing_index, training_hours, as these did not seem to improve the algorithim

df.drop('enrollee_id', inplace=True, axis=1)
df.drop('city', inplace=True, axis=1)
df.drop('city_development_index', inplace=True, axis=1)
df.drop('training_hours', inplace=True, axis=1)
print(df)

      gender      relevent_experience  ... last_new_job target
0       Male  Has relevent experience  ...            1      1
1       Male   No relevent experience  ...           >4      0
2        NaN   No relevent experience  ...        never      0
3        NaN   No relevent experience  ...        never      1
4       Male  Has relevent experience  ...            4      0
...      ...                      ...  ...          ...    ...
19153   Male   No relevent experience  ...            1      1
19154   Male  Has relevent experience  ...            4      1
19155   Male  Has relevent experience  ...            4      0
19156   Male  Has relevent experience  ...            2      0
19157    NaN   No relevent experience  ...            1      0

[19158 rows x 10 columns]


In [6]:
#Hot-Encoder

#https://datagy.io/pandas-get-dummies/

categorical_columns = ['gender', 'relevent_experience', 'enrolled_university', 'education_level', 'major_discipline', 'experience', 'company_size', 'company_type', 'last_new_job' ]    
numerical_columns = ['city_development_index', 'training_hours',]

#hot-encodes and merges back into the df, as a vector
  #It works by creating each item variation, into an array. So if category is dog, cat, fish , then,  cat= <0,10>  ;  dog= <1,0,0>   ; fish =<0,0,1>
for column in categorical_columns:  
    tempdf = pd.get_dummies(df[column], prefix=column)
    df = pd.merge(
        left=df,
        right=tempdf,
        left_index=True,
        right_index=True,
    )
    df = df.drop(columns=column)
print(df)

       target  gender_Female  ...  last_new_job_>4  last_new_job_never
0           1              0  ...                0                   0
1           0              0  ...                1                   0
2           0              0  ...                0                   1
3           1              0  ...                0                   1
4           0              0  ...                0                   0
...       ...            ...  ...              ...                 ...
19153       1              0  ...                0                   0
19154       1              0  ...                0                   0
19155       0              0  ...                0                   0
19156       0              0  ...                0                   0
19157       0              0  ...                0                   0

[19158 rows x 62 columns]


In [ ]:
#shows all the unique possible values for a feature
#df['gender'].unique()

In [13]:
#Outputs the rows, and cols in the dataframe
df.shape

(19158, 62)

In [14]:
#Used to better visualize the data
%load_ext google.colab.data_table
# %unload_ext google.colab.data_table  #disables it
df.head(150)  #used to display all the headers of the csv file

,target,gender_Female,gender_Male,gender_Other,relevent_experience_Has relevent experience,relevent_experience_No relevent experience,enrolled_university_Full time course,enrolled_university_Part time course,enrolled_university_no_enrollment,education_level_Graduate,education_level_High School,education_level_Masters,education_level_Phd,education_level_Primary School,major_discipline_Arts,major_discipline_Business Degree,major_discipline_Humanities,major_discipline_No Major,major_discipline_Other,major_discipline_STEM,experience_1,experience_10,experience_11,experience_12,experience_13,experience_14,experience_15,experience_16,experience_17,experience_18,experience_19,experience_2,experience_20,experience_3,experience_4,experience_5,experience_6,experience_7,experience_8,experience_9,experience_<1,experience_>20,company_size_100-500,company_size_1000-4999,company_size_10000+,company_size_50-99,company_size_500-999,company_size_5000-9999,company_size_<10,company_size_Oct-49,company_type_Early Stage Startup,company_type_Funded Startup,company_type_NGO,company_type_Other,company_type_Public Sector,company_type_Pvt Ltd,last_new_job_1,last_new_job_2,last_new_job_3,last_new_job_4,last_new_job_>4,last_new_job_never
0,1,0,1,0,1,0,0,0,1,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
1,0,0,1,0,0,1,0,0,1,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0
2,0,0,0,0,0,1,1,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
3,1,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1
4,0,0,1,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145,0,0,0,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
146,0,0,0,0,1,0,0,0,1,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
147,1,0,1,0,1,0,0,0,1,1,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
148,0,0,1,0,1,0,0,1,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1


In [15]:
#Used to get statistics about each column
df.describe()

,target,gender_Female,gender_Male,gender_Other,relevent_experience_Has relevent experience,relevent_experience_No relevent experience,enrolled_university_Full time course,enrolled_university_Part time course,enrolled_university_no_enrollment,education_level_Graduate,education_level_High School,education_level_Masters,education_level_Phd,education_level_Primary School,major_discipline_Arts,major_discipline_Business Degree,major_discipline_Humanities,major_discipline_No Major,major_discipline_Other,major_discipline_STEM,experience_1,experience_10,experience_11,experience_12,experience_13,experience_14,experience_15,experience_16,experience_17,experience_18,experience_19,experience_2,experience_20,experience_3,experience_4,experience_5,experience_6,experience_7,experience_8,experience_9,experience_<1,experience_>20,company_size_100-500,company_size_1000-4999,company_size_10000+,company_size_50-99,company_size_500-999,company_size_5000-9999,company_size_<10,company_size_Oct-49,company_type_Early Stage Startup,company_type_Funded Startup,company_type_NGO,company_type_Other,company_type_Public Sector,company_type_Pvt Ltd,last_new_job_1,last_new_job_2,last_new_job_3,last_new_job_4,last_new_job_>4,last_new_job_never
count,19158.000000,19158.000000,19158.000000,19158.000000,19158.000000,19158.000000,19158.000000,19158.000000,19158.000000,19158.000000,19158.000000,19158.000000,19158.000000,19158.000000,19158.000000,19158.000000,19158.000000,19158.000000,19158.000000,19158.000000,19158.000000,19158.000000,19158.000000,19158.000000,19158.000000,19158.000000,19158.000000,19158.000000,19158.000000,19158.000000,19158.000000,19158.000000,19158.000000,19158.000000,19158.000000,19158.000000,19158.000000,19158.000000,19158.000000,19158.000000,19158.000000,19158.000000,19158.000000,19158.000000,19158.000000,19158.000000,19158.000000,19158.000000,19158.000000,19158.000000,19158.000000,19158.000000,19158.000000,19158.000000,19158.000000,19158.000000,19158.000000,19158.000000,19158.000000,19158.000000,19158.000000,19158.000000
mean,0.249348,0.064621,0.690103,0.009970,0.719908,0.280092,0.196106,0.062533,0.721213,0.605387,0.105282,0.227633,0.021610,0.016077,0.013206,0.017069,0.034920,0.011640,0.019887,0.756446,0.028656,0.051415,0.034659,0.025786,0.020827,0.030588,0.035807,0.026516,0.017852,0.014615,0.015868,0.058827,0.007725,0.070675,0.073233,0.074642,0.063472,0.053659,0.041862,0.051154,0.027247,0.171521,0.134200,0.069318,0.105387,0.160925,0.045777,0.029387,0.068274,0.076783,0.031475,0.052250,0.027195,0.006316,0.049849,0.512423,0.419668,0.151373,0.053450,0.053711,0.171730,0.127988
std,0.432647,0.245861,0.462463,0.099352,0.449056,0.449056,0.397060,0.242127,0.448414,0.488780,0.306925,0.419316,0.145409,0.125774,0.114159,0.129530,0.183582,0.107262,0.139617,0.429238,0.166843,0.220848,0.182920,0.158499,0.142808,0.172202,0.185815,0.160669,0.132415,0.120010,0.124968,0.235306,0.087555,0.256289,0.260526,0.262820,0.243817,0.225350,0.200280,0.220317,0.162807,0.376973,0.340876,0.254001,0.307059,0.367471,0.209007,0.168894,0.252223,0.266253,0.174602,0.222536,0.162655,0.079223,0.217638,0.499859,0.493517,0.358421,0.224935,0.225453,0.377155,0.334086
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000

In [8]:
#dataset = tf.data.Dataset.from_tensor_slices((df.values, target.values))
  #randomizes the data as well
#Splits the dataframe into the 3 datasets
#https://stackoverflow.com/questions/38250710/how-to-split-data-into-3-sets-train-validation-and-test
train, validate, test = \
              np.split(df.sample(frac=1, random_state=42), 
                       [int(.7*len(df)), int(.85*len(df))])
              
print(train)
print(validate)
print(test)

       target  gender_Female  ...  last_new_job_>4  last_new_job_never
16144       0              0  ...                0                   0
13661       0              1  ...                0                   0
344         0              0  ...                0                   0
1034        0              0  ...                0                   0
8109        0              0  ...                0                   0
...       ...            ...  ...              ...                 ...
14767       0              0  ...                0                   0
8106        0              0  ...                0                   0
1893        0              0  ...                0                   0
7833        0              0  ...                0                   1
18591       0              0  ...                0                   0

[13410 rows x 62 columns]
       target  gender_Female  ...  last_new_job_>4  last_new_job_never
11587       1              0  ...                0

In [ ]:
#train_dataset = dataset.shuffle(len(df)).batch(1)  #increasing batch size, makes it runner faster, since instad of 1 large batch to stream into it,
                                                    #we can do multiple, smaller ones, thus can run on RAM, ie faster
                                                    #note: epoch: means one stream the full dataset
                                                    #so more epochs, the more it will "understand" the data set
                                                      #note: too many & it'll begin to memorize it tho

In [ ]:
#Used to count the size of the dataset, so that it that it can later divide the dataset
#i = 0
#for elem in full_dataset: 
#  i = i + 1
#print(i)
#total_dataset_size = i;

In [ ]:
#print(type(dataset))
#https://stackoverflow.com/questions/48213766/split-a-dataset-created-by-tensorflow-dataset-api-in-to-train-and-test/51258695
#1st 3 lines get the size of the dataset for each type
#training_size = int(0.7 * total_dataset_size)
#print("training_size = ", training_size)

#testing_size = int(0.15 * total_dataset_size)
#print("testing_size = ", testing_size)

#validating_size = int(0.15 * total_dataset_size)
#print("validating_size = ", validating_size)

In [ ]:
#Sets up the train/test/validate datasets
#full_dataset = dataset.shuffle(len(df)).batch(1)   

#full_dataset = tf.data.TFRecordDataset(df)
 
#training_dataset = full_dataset.take(training_size)  #takes 1st 80%
#print(training_dataset)

#testing_dataset = full_dataset.skip(training_size) 

#validating_dataset = testing_dataset.skip(validating_size)
#print(validating_dataset)

#testing_dataset = testing_dataset.take(testing_size)
#print(testing_dataset)

In [ ]:
test.shape

In [17]:
# ensure all data are floating point values
test = test.astype('uint8')

In [ ]:
#Used to determine how many neurons to use in the input layer
n_features = test.shape[1]
print(n_features)

In [9]:
#split the train into data that knows the TARGET, and one that doesnt
#-----
#https://www.kaggle.com/jamesleslie/titanic-neural-network-for-beginners

#adpated the above for my dataframe:
X_train = train[pd.notnull(train['target'])].drop(['target'], axis=1)  #training
y_train = train[pd.notnull(train['target'])]['target']                 #validate
X_test = test[pd.isnull(test['target'])].drop(['target'], axis=1)      #test

print(X_train)
print(y_train)
print(X_test)

       gender_Female  gender_Male  ...  last_new_job_>4  last_new_job_never
16144              0            0  ...                0                   0
13661              1            0  ...                0                   0
344                0            0  ...                0                   0
1034               0            1  ...                0                   0
8109               0            1  ...                0                   0
...              ...          ...  ...              ...                 ...
14767              0            1  ...                0                   0
8106               0            0  ...                0                   0
1893               0            0  ...                0                   0
7833               0            0  ...                0                   1
18591              0            1  ...                0                   0

[13410 rows x 61 columns]
16144    0
13661    0
344      0
1034     0
8109     0
      

In [16]:
def get_compiled_model():
  model = tf.keras.Sequential([

    tf.keras.layers.Dense(61, input_shape=(61,), use_bias=True, bias_initializer='zeros',),  #input (1 neuron per column)
    tf.keras.layers.Dense(30, activation='tanh'),  #hidden
    tf.keras.layers.Dense(15, activation='tanh'),   #hidden
    tf.keras.layers.Dense(1, activation='tanh')  #ouput layer
  ])


  model.compile(optimizer='adam', #adam algorithim computes the gradient descent
                loss=tf.keras.losses.BinaryCrossentropy(from_logits=True), 
                metrics=['accuracy'])
  return model

In [17]:
model = get_compiled_model()
print(model)

training = model.fit(X_train, y_train, epochs=37, batch_size=5, validation_split=0.75, verbose=1)

Epoch 1/37
671/671 [==============================] - 4s 5ms/step - loss: 0.5631 - accuracy: 0.7521 - val_loss: 0.5470 - val_accuracy: 0.7521
Epoch 2/37
671/671 [==============================] - 3s 5ms/step - loss: 0.5451 - accuracy: 0.7555 - val_loss: 0.5449 - val_accuracy: 0.7550
Epoch 3/37
671/671 [==============================] - 3s 4ms/step - loss: 0.5400 - accuracy: 0.7617 - val_loss: 0.5404 - val_accuracy: 0.7519
Epoch 4/37
671/671 [==============================] - 3s 4ms/step - loss: 0.5290 - accuracy: 0.7615 - val_loss: 0.5407 - val_accuracy: 0.7520
Epoch 5/37
671/671 [==============================] - 3s 4ms/step - loss: 0.5272 - accuracy: 0.7671 - val_loss: 0.5398 - val_accuracy: 0.7520
Epoch 6/37
671/671 [==============================] - 3s 4ms/step - loss: 0.5220 - accuracy: 0.7659 - val_loss: 0.5416 - val_accuracy: 0.7597
Epoch 7/37
671/671 [==============================] - 3s 5ms/step - loss: 0.5246 - accuracy: 0.7740 - val_loss: 0.5441 - val_accuracy: 0.7525
Epoch 

In [18]:
# evaluates the model
eval_scores = model.evaluate(X_train, y_train, verbose=2)
print("Test loss:", eval_scores[0])
print("Test accuracy:", eval_scores[1])

420/420 - 0s - loss: 0.5459 - accuracy: 0.7644
Test loss: 0.5459156036376953
Test accuracy: 0.76442950963974


In [19]:
# make a prediction
predictions = model.predict(X_train)
print(predictions)

[[-0.9999962 ]
 [-0.9998    ]
 [-0.99994934]
 ...
 [-0.8745967 ]
 [-0.9999472 ]
 [-0.8513872 ]]


In [28]:
from google.colab import drive
drive.mount('drive')

Mounted at drive


SyntaxError: ignored

In [29]:
df.to_csv('data.csv')
!cp data.csv "drive/My Drive/"